In [ ]:
import pandas as pd
import numpy as np
import ast
import textwrap

import copy


import plotly.graph_objects as go
from plotly.subplots import make_subplots

import spacy
from bertopic import BERTopic

#Representación de topics
from bertopic.vectorizers import ClassTfidfTransformer
#Embedding model
from sentence_transformers import SentenceTransformer
#UMAP para reducir la dimensionalidad
from umap import UMAP
#clusterizador
from hdbscan import HDBSCAN
#Mapa de texto a vector 
from sklearn.feature_extraction.text import CountVectorizer
#representacion fine-tuned de bert topic: https://maartengr.github.io/BERTopic/index.html#fine-tune-topic-representations
from bertopic.representation import KeyBERTInspired

# Objetivo

Se tienen unas vacantes laborales para el sector energético. Cada vacante tienen un listado de conocimientos. Se quiere agrupar los conocimientos en categorias estandar para discernir cuales son los conocimientos que está demandando el mercado laboral.

## Referencias

https://maartengr.github.io/BERTopic/index.html#variations

## Funciones

In [2]:
def safe_parse_conocimientos(value):
    '''
    Cambia de formato las entradas de la forma
    "'palabra_1','palabra_2',..., 'palabra_n'"
    a
    ['palabra_1','palabra_2,...,'palabra_n']
    '''
    try:
        # Si no está entre corchetes, lo envolvemos
        if not value.strip().startswith("["):
            value = f"[{value}]"
        return ast.literal_eval(value)
    except Exception:
        return [str(value)]  # Si no se puede evaluar, lo devolvemos como lista con un solo string


## Leer datos

In [3]:
conoc_df=pd.read_excel("../data/conoc_tej.xlsx",index_col=False)
conoc_df=conoc_df.replace('NA',np.nan)

In [4]:
print(f"Numero de filas: {conoc_df.shape[0]} \nNumero de columnas: {conoc_df.shape[1]}")
conoc_df.head()

Numero de filas: 3087 
Numero de columnas: 26


,id,titulo_vacante,descripcion,tipo_documento,cod_depto,fecha_inicio,fecha_cierre,registro_mes,registro_ano,tipo1prestador,...,depto,mpio,rangos_salariales,numero_documento_anom,codigo,descripcion_input,profesion,educacion,conocimientos,clasif_cine_nombre
0,180,auxiliar de servicio tecnico industrial,importante multinacional del sector energetico...,NI,11,44942,30/01/2023,1,2023,Privado,...,"BOGOTA, D. C.",BOGOTA D.C.,$1.500.001 - $2.000.000,fdd3b5bff414026a8d0b7c09bc232281d192797cec6b2b...,OFERTA_181,auxiliar de servicio tecnico industrial; impor...,'Auxiliar servicio técnico industrial',"'Mecánico', 'Electromecánico'","'mantenimiento de maquinaria industrial', 'man...",71.0-Ingeniería y profesiones afines
1,368,tecnico fumigador - con moto,importante empresa dedicada a la prestacion de...,NI,11,44945,30/01/2023,1,2023,Privado,...,"BOGOTA, D. C.",BOGOTA D.C.,$1.000.001 - $1.500.000,fdd3b5bff414026a8d0b7c09bc232281d192797cec6b2b...,OFERTA_369,tecnico fumigador - con moto; importante empre...,'Técnico fumigador','Bachiller',"'Control integrado de plagas', 'Control de roe...",1.0-Programas y certificaciones básicos
2,637,ingeniero preventa,importante empresa requiere para su equipo de ...,NI,11,44950,30/01/2023,1,2023,Privado,...,"BOGOTA, D. C.",BOGOTA D.C.,$2.000.001 - $3.000.000,f1bb3499cfe1370bbf73106827e6ba0c3b6fe5c06db679...,OFERTA_638,ingeniero preventa; importante empresa requier...,'Ingeniero preventa',"'Ingeniería electrónica', 'Mecatrónica', 'Tele...","'Proyectos de seguridad electrónica', 'Dimensi...",41.0-Educación comercial y administración
3,702,tecnico electricista - lugar de trabajo: bogota,importante empresa requiere tecnico electricis...,NI,11,44936,30/01/2023,1,2023,Privado,...,"BOGOTA, D. C.",BOGOTA D.C.,$1.000.001 - $1.500.000,53547f1b0955425b627d74c9865433f6ef9137a8ecb8c0...,OFERTA_703,tecnico electricista - lugar de trabajo: bogot...,'Técnico electricista','NO_APLICA','Mantenimiento preventivo y correctivo de rede...,1.0-Programas y certificaciones básicos
4,1123,vendedor/a - experiencia en ventes telefonicas,importante compania del sector de saneamiento ...,NI,11,44942,17/03/2023,1,2023,Privado,...,"BOGOTA, D. C.",BOGOTA D.C.,$1.000.001 - $1.500.000,e1c11017d0aff28b84b75cef8b7a4d5ae5e5f140530b9c...,OFERTA_1124,vendedor/a - experiencia en ventes telefonicas...,'Vendedor','NO_APLICA',"'Ventas telefónicas', 'Call center'",1.0-Programas y certificaciones básicos


In [5]:
#descripcion de la primera vacante
wrapped_text = textwrap.fill(conoc_df['descripcion'][0], width=80)
print(wrapped_text)

importante multinacional del sector energetico requiere para su equipo de
trabajo tecnico o tecnologo mecanico, electromecanico o a fines, con minimo tres
anos de experiencia en el desarrollo de trabajos asociados con el mantenimiento
de maquinaria industrial (no locativo), manejo de bombas de vacio, equipos de
soldadura, mecanica hidraulica. te ofrecemosii salario:2.000.000 + prestaciones
de ley horario de lunes a sabado - disponibilidad domingo o festivos lugar de
trabajo tenjo cundinamarca contrato obra o labor 12 meses beneficio de casino y
ruta


In [6]:
#Conocimientos de la vacante
conoc_df['conocimientos'][0]

"'mantenimiento de maquinaria industrial', 'manejo de bombas de vacío', 'equipos de soldadura', 'mecánica hidráulica'"

In [7]:
#Generamos una lista con los conocimientos individuales
#En esta version, cada documento es un conocimiento especifico
corpus = conoc_df['conocimientos'].apply(safe_parse_conocimientos).sum()
print(f"Numero de documentos en el corpus:{len(corpus)}")
#En esta version, cada "documento" es un listado de conocimientos
#conocimientos = conoc_df.conocimientos.to_list()

Numero de documentos en el corpus:16508


## Instanciar modelo

In [8]:
#spamsih stop words
nlp = spacy.blank("es")
spanish_stopwords = nlp.Defaults.stop_words
del(nlp)

c-TF-IDF will show you what words are most important for each topic, relative to the other topics in the corpus. This allows you to identify topic-specific terms that help characterize the content of each topic.

In [56]:
#Extract embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
#UMAP para reducir la dimensionalidad de los embeddings
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=123)
#Clusterizador de emebeddings
hdbscan_model = HDBSCAN(min_cluster_size=50, metric='euclidean', cluster_selection_method='eom', prediction_data=True)
#Tokenizador de topicos
vectorizer_model = CountVectorizer(stop_words=list(spanish_stopwords), min_df=2, ngram_range=(1,3))
#Create topic representation
ctfidf_model = ClassTfidfTransformer()
#representacion fine-tuned de bert topic:
representation_model = KeyBERTInspired()


model = BERTopic(
    embedding_model=embedding_model,            # Step 1 - Extract embeddings
    umap_model=umap_model,                      # Step 2 - Reduce dimensionality
    hdbscan_model=hdbscan_model,                # Step 3 - Cluster reduced embeddings
    vectorizer_model=vectorizer_model,          # Step 4 - Tokenize topics
    ctfidf_model=ctfidf_model,                  # Step 5 - Extract topic words
    representation_model=representation_model,  # Step 6 - (Optional) Fine-tune topic representations
    language= "spanish"
    )#"spanish"#"multilingual


#model = BERTopic(vectorizer_model=vectorizer_model,umap_model=umap_model, representation_model=representation_model,language= "spanish")#"spanish"#"multilingual

topics, probabilities = model.fit_transform(corpus)

# Make a deep copy of the model
model_copy = copy.deepcopy(model)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [58]:
print(f"Numero de topicos: {len(model.get_topics())}")
print("\nResumen de la cardinalidad:")
model.get_topic_freq()[['Count']].describe()

Numero de topicos: 94

Resumen de la cardinalidad:


,Count
count,94.000000
mean,175.617021
std,587.664927
min,52.000000
25%,65.250000
50%,88.500000
75%,130.750000
max,5745.000000


In [59]:
#outliers
print("Outliers:\n")
print("Palabras clave en el topico outlier (-1):")
model.get_topic( -1 )

Outliers:

Palabras clave en el topico outlier (-1):


[('paquete office', np.float32(0.9078065)),
 ('paquete', np.float32(0.7176398)),
 ('office', np.float32(0.5236668)),
 ('soporte', np.float32(0.48858148)),
 ('soporte técnico', np.float32(0.46840516)),
 ('sistemas', np.float32(0.4650221)),
 ('corte', np.float32(0.45178977)),
 ('sistema', np.float32(0.44529915)),
 ('pruebas', np.float32(0.43587822)),
 ('sistemas integrados gestión', np.float32(0.41668606))]

# Visualizar

In [76]:
model.visualize_topics()

In [16]:
model.visualize_barchart()

# Reduce Topics

In [ ]:
#reduce topics
new_model = model_copy.reduce_topics(corpus, nr_topics="auto", use_ctfidf=True)
new_topics, new_probs = new_model.transform(corpus)


In [63]:
print(f"Numero de topicos: {len(new_model.get_topics())}")
print("\nResumen de la cardinalidad:")
new_model.get_topic_freq()[['Count']].describe()

Numero de topicos: 64

Resumen de la cardinalidad:


,Count
count,64.000000
mean,257.937500
std,803.693831
min,53.000000
25%,65.750000
50%,87.000000
75%,124.000000
max,5745.000000


In [64]:
#outliers
print("Outliers:\n")
new_model.get_topic( -1 )

Outliers:



[('paquete office', np.float32(0.902567)),
 ('paquete', np.float32(0.70241714)),
 ('office', np.float32(0.553205)),
 ('soporte', np.float32(0.54059243)),
 ('soporte técnico', np.float32(0.5012655)),
 ('sostenibilidad', np.float32(0.4725801)),
 ('sistemas', np.float32(0.46312279)),
 ('salud ocupacional', np.float32(0.45154107)),
 ('corte', np.float32(0.45051685)),
 ('seguimiento', np.float32(0.444187))]

In [69]:
new_model.visualize_topics()

# Reduce Outliers

In [ ]:
#Numero de topicos outliers en model 
outlier_count = np.sum(np.array(topics) == -1)

print(f"Número de outliers model: {outlier_count}")

#Numero de topicos outliers en new_model 
outlier_count = np.sum(np.array(new_topics) == -1)

print(f"Número de outliers new_model (modelo con reduccion de topicos): {outlier_count}")

Número de outliers: 5745


In [ ]:
#Reduce outliers
reduced_topics_model = model_copy.reduce_outliers(corpus, topics)

#Update the model
model_copy.update_topics(corpus,topics=reduced_topics_model)
model_copy.fit(corpus)

# Visualize

In [81]:
fig1 = model.visualize_topics()
fig2 = model_copy.visualize_topics()

# Create a subplot: 1 row, 2 columns
combined_fig = make_subplots(rows=1, cols=2, subplot_titles=("Original Model", "Model After Reducing Outliers"))

# Add traces from the first figure
for trace in fig1.data:
    combined_fig.add_trace(trace, row=1, col=1)

# Add traces from the second figure
for trace in fig2.data:
    combined_fig.add_trace(trace, row=1, col=2)

# Update layout if you want
combined_fig.update_layout(height=600, width=1200, showlegend=False)

# Show combined figure
combined_fig.show()

# Clasificación Jerárquica

In [82]:
from scipy.cluster import hierarchy as sch

In [83]:
linkage_function = lambda x: sch.linkage(x, 'single', optimal_ordering=True)

model_jer= model.hierarchical_topics(corpus, linkage_function = linkage_function)
model_jer_refit= model_copy.hierarchical_topics(corpus, linkage_function = linkage_function)

100%|██████████| 92/92 [00:00<00:00, 302.79it/s]


In [ ]:
# Generate both hierarchy figures
fig1 = model_copy.visualize_hierarchy(hierarchical_topics=model_jer_refit)
fig2 = model.visualize_hierarchy(hierarchical_topics=model_jer)

# Create a subplot: 1 row, 2 columns
combined_fig = make_subplots(rows=1, cols=2, subplot_titles=("Model after Reducing Outliers", "Original Model"))

# Add traces from the first hierarchy figure
for trace in fig1.data:
    combined_fig.add_trace(trace, row=1, col=1)

# Add traces from the second hierarchy figure
for trace in fig2.data:
    combined_fig.add_trace(trace, row=1, col=2)

# Update layout
combined_fig.update_layout(height=600, width=1200, showlegend=False)

# Show combined figure
combined_fig.show()

--------